<a href="https://colab.research.google.com/github/greyhound101/brats/blob/main/patient_wise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install medpy

In [ ]:
from tqdm import tqdm
from medpy.io import load, save
import numpy as np
import gc
import cv2
import glob
target_dict={'HGG':1,'LGG':0}
targets_h=[]
images_h=[]
for path in tqdm(glob.glob('../input/brain-tumor-segmentation-brats-2019/MICCAI_BraTS_2019_Data_Training/HGG/*/*_flair.nii')):
    trg=path.split('/')
    img=load(path)[0]
    seg=load(path.replace('_flair','_seg'))[0]
    mn=np.mean(img)
    std=np.std(img)
    try:
        pos=round((min(np.where(seg==1)[-1])+max(np.where(seg==1)[-1]))/2)
        img=cv2.resize(img[:,:,pos], (224,224), interpolation = cv2.INTER_AREA )
        img=(img-mn)/std
        img=np.repeat(np.expand_dims(img,-1),3,-1)
        images_h.append(img)
        targets_h.append(target_dict[path.split('/')[4]])
        del([img,seg])
        gc.collect()
    except:
        continue

In [ ]:
from tqdm import tqdm
from medpy.io import load, save
import numpy as np
import gc
import cv2
import glob
target_dict={'HGG':1,'LGG':0}
targets_l=[]
images_l=[]
for path in tqdm(glob.glob('../input/brain-tumor-segmentation-brats-2019/MICCAI_BraTS_2019_Data_Training/LGG/*/*_flair.nii')):
    image=np.zeros((224,224,9))
    trg=path.split('/')
    original=load(path)[0]
    seg=load(path.replace('_flair','_seg'))[0]
    mn=np.mean(original)
    std=np.std(original)
    try:
        pos=round((min(np.where(seg==1)[-1])+max(np.where(seg==1)[-1]))/2)
        dev=np.std(list(range(original.shape[-1])))*0.08
        
        img=cv2.resize(original[:,:,pos+int(dev)], (224,224), interpolation = cv2.INTER_AREA )
        img=(img-mn)/std
        img=np.repeat(np.expand_dims(img,-1),3,-1)
        image[:,:,:3]=img
        targets_l.append(target_dict[path.split('/')[4]])
        
        img=cv2.resize(original[:,:,pos-int(dev)], (224,224), interpolation = cv2.INTER_AREA )
        img=(img-mn)/std
        img=np.repeat(np.expand_dims(img,-1),3,-1)
        image[:,:,3:6]=img
        
        
        img=cv2.resize(original[:,:,pos], (224,224), interpolation = cv2.INTER_AREA )
        img=(img-mn)/std
        img=np.repeat(np.expand_dims(img,-1),3,-1)
        image[:,:,6:9]=img
        images_l.append(image)
        
        del([img,seg])
        gc.collect()
    except:
        continue

In [ ]:


targets=targets_h+targets_l
targets=np.array(targets)

np.save('targets_h.npy',targets_h)
np.save('targets_l.npy',targets_l)
np.save('images_h.npy',images_h)
np.save('images_l.npy',images_l)

from tensorflow.keras.layers import *
import tensorflow as tf
import random, os, sys
import numpy as np
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.initializers import *
import tensorflow as tf
from tensorflow.keras.applications import *
from tensorflow.keras.layers import *
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow import keras
from tensorflow.keras import backend as K
class LayerNormalization(Layer):
    def __init__(self, eps=1e-6, **kwargs):
        self.eps = eps
        super(LayerNormalization, self).__init__(**kwargs)
    def build(self, input_shape):
        self.gamma = self.add_weight(name='gamma', shape=input_shape[-1:],
                                     initializer=Ones(), trainable=True)
        self.beta = self.add_weight(name='beta', shape=input_shape[-1:],
                                    initializer=Zeros(), trainable=True)
        super(LayerNormalization, self).build(input_shape)
    def call(self, x):
        mean = K.mean(x, axis=-1, keepdims=True)
        std = K.std(x, axis=-1, keepdims=True)
        return self.gamma * (x - mean) / (std + self.eps) + self.beta
    def compute_output_shape(self, input_shape):
        return input_shape
class abc(keras.layers.Layer):
    def __init__(self,
                 hidden_dim,
                 **kwargs):
        self.hidden_dim = hidden_dim
        
        self.conv1=Conv1D(self.hidden_dim,1)
        self.conv2=Conv1D(self.hidden_dim,1)
        self.conv3=Conv1D(self.hidden_dim,1)
        
        self.Wq = self.Wk = self.Wv = self.Wo = None
        self.bq = self.bk = self.bv = self.bo = None

        self.intensity = self.attention = None
        super(abc, self).__init__(**kwargs)

    def call(self, inputs, mask=None):
        
        q, k, v = inputs
        
        q=self.conv1(q)
        k=self.conv2(k)
        v=self.conv3(v)
        
        
        def scaled_dot_product_attention(inputs):
          query, key, value = inputs
          feature_dim = K.shape(query)[-1]
          e = K.batch_dot(query, key, axes=2) 
          intensity = e
          e = K.exp(e - K.max(e, axis=-1, keepdims=True))
          attention = e / K.sum(e, axis=-1, keepdims=True)
          v = K.batch_dot(attention, value)
          return v
       
       
        y = scaled_dot_product_attention(inputs=[q,k,v])
        
        
        return y

from tensorflow import keras 
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import *
from tensorflow.keras.applications import *
def load_model():   
  
  K.clear_session() 
  mod=densenet.DenseNet121(include_top=True, weights='imagenet')
  d = mod.get_layer('conv5_block16_concat').output

  inp = mod.get_layer('conv3_block12_concat').output
  a = Reshape((28*28,512))(inp)
  d_a = keras.layers.UpSampling2D(interpolation='bilinear',size=(4,4))(d)
  d_a = Reshape((28*28,1024))(d_a)
  a = abc(hidden_dim=512)([a,d_a,a])
  a = LayerNormalization()(a)
  a = Reshape((28,28,512,))(a)
  a = keras.layers.GlobalAveragePooling2D()(a)

  inp = mod.get_layer('conv4_block24_concat').output
  b = Reshape((14*14,1024))(inp)
  d_b = keras.layers.UpSampling2D(interpolation='bilinear',size=(2,2))(d)
  d_b = Reshape((14*14,1024))(d_b)
  b = abc(hidden_dim=1024)([b,d_b,b])
  b = LayerNormalization()(b)
  b = Reshape((14,14,1024,))(b)
  b = keras.layers.GlobalAveragePooling2D()(b)

  d = keras.layers.GlobalAveragePooling2D()(d)
  

  b = Dense(3, activation="softmax")(b) 
  b = Reshape((-1,3))(b) 
  a = Dense(3, activation="softmax")(a) 
  a = Reshape((-1,3))(a) 
  d = Dense(3, activation="softmax")(d) 
  d = Reshape((-1,3))(d) 
  
  conc=Concatenate(axis=1)([a,b,d])
  conc=keras.layers.GlobalAveragePooling1D()(conc)
  mod=Model(inputs=mod.input,outputs=conc)
  mod.load_weights('../input/attention/weights.hdf5')
  for layer in mod.layers:
    layer.trainable=False
  a=mod.layers[-9].output
  b=mod.layers[-10].output
  d=mod.layers[-11].output
  b = Dense(1, activation="sigmoid")(b) 
  b = Reshape((-1,1))(b) 
  a = Dense(1, activation="sigmoid")(a) 
  a = Reshape((-1,1))(a) 
  d = Dense(1, activation="sigmoid")(d) 
  d = Reshape((-1,1))(d) 
  conc=Concatenate(axis=1)([a,b,d])
  conc=keras.layers.GlobalAveragePooling1D()(conc)
  mod=Model(inputs=mod.input,outputs=conc)
  for en,layer in enumerate(mod.layers):
    if layer.trainable==True:
        print(en)
  return mod

mod=load_model()



In [ ]:


from imgaug import augmenters as iaa
def rotate_image(image, angle):
  image_center = tuple(np.array(image.shape[1::-1]) / 2)
  rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
  result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
  return result
def Hflip( images):
		seq = iaa.Sequential([iaa.Fliplr(1.0)])
		return seq.augment_images(images)
def Vflip( images):
		seq = iaa.Sequential([iaa.Flipud(1.0)])
		return seq.augment_images(images)
def noise(images):
    ls=[]
    for i in images:
        x = np.random.normal(loc=0, scale=0.05, size=(299,299,3))
        ls.append(i+x)
    return ls
def rotate(images):
    ls=[]
    for angle in range(-15,20,5):
        for image in images:
            ls.append(rotate_image(image,angle))
    return ls

class DataGenerator(keras.utils.Sequence):
  def __init__(self, images, labels, batch_size=64, image_dimensions = (96 ,96 ,3), shuffle=False, augment=False):
    self.labels       = labels              # array of labels
    self.images = images        # array of image paths
    self.batch_size   = batch_size          # batch size
    self.on_epoch_end()

  def __len__(self):
    return int(np.floor(self.labels.shape[0] / self.batch_size))

  def on_epoch_end(self):
    self.indexes = np.arange(self.labels.shape[0])

  def __getitem__(self, index):
		# selects indices of data for next batch
    indexes = self.indexes[index * self.batch_size : (index + 1) * self.batch_size]
    # select data and load images
    labels = self.labels[indexes]
    img = [self.images[k].astype(np.float32) for k in indexes]
    imgH=Hflip(img)
    imgV=Vflip(img)
    imgR=rotate(img)
    images=[]
    images.extend(imgH)
    images.extend(imgV)
    images.extend(imgR)
    lbl=labels.copy()
    labels=np.repeat(labels,9)
    del([imgV,imgR,imgH,lbl])
    gc.collect()
    #images = np.array([preprocess_input(img) for img in images])
    return np.asarray(images).astype('float64'), labels.astype('uint8')

import cv2
import gc
from tensorflow.keras.optimizers import *
from sklearn.model_selection import *
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score
kf=KFold(n_splits=5)
for train_index, test_index in kf.split(targets):
  mod=load_model()  
  X_train, X_test, y_train, y_test = [],[],[],[]
  for i in train_index:
    if i<len(targets_h):
        X_train.append(images_h[i])
        y_train.append(1)
    else:
        X_train.append(images_l[i-len(targets_h)][:,:,:3])
        X_train.append(images_l[i-len(targets_h)][:,:,3:6])
        X_train.append(images_l[i-len(targets_h)][:,:,6:9])
        y_train.extend([0,0,0])
  for i in test_index:
    if i<len(targets_h):
        X_test.append(images_h[i])
        y_test.append(1)
    else:
        X_test.append(images_l[i-len(targets_h)][:,:,:3])
        X_test.append(images_l[i-len(targets_h)][:,:,3:6])
        X_test.append(images_l[i-len(targets_h)][:,:,6:9])
        y_test.extend([0,0,0])
  X_train=np.asarray(X_train)
  X_test=np.asarray(X_test)
  y_train=np.asarray(y_train)
  y_test=np.asarray(y_test)
  print(X_train.shape, X_test.shape,y_train.shape,y_test.shape)
  train_data = DataGenerator(X_train,y_train, batch_size=4, augment=True)
  mod.compile(optimizer=Adam(5e-5,decay=1e-4), loss='binary_crossentropy', metrics=['accuracy'])
  print('started training')
  hist=mod.fit_generator(train_data,epochs=50,verbose=0)
  plt.plot(hist.history['loss'])
  plt.show()
  pre=mod.predict(X_test)
  print(accuracy_score(np.round(pre),y_test))
  del([mod,X_train, X_test, y_train, y_test,train_data])
  gc.collect()

